In [1]:
import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()
import os
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords

In [78]:
#98
tag_pol = ['#KeikoNova', '#keikonuncamas', '#FujimoriNuncaMás', '#NoAKeiko',
       '#Keikino', '#KeikoTAPERdiendo', '#GolpeElectoral',
       '#FuerzaPopular', '#ConMiVotoNoTeMetas',
       '#BICENTENARIOMEJORCONSTITUCION', '#FujimafiaNuncaMás',
       '#NoAlNarcoEstado', '#FujimorismoNuncaMas',
       '#KeikoGolpistaYDictadora', '#keikonoimpugnesmiacta',
       '#KEIKOMENTIROSA', '#KeikoPresidenta2021', '#KeikoPresidenta',
       '#PedroCastilloPresidente', '#PedroCastilloNoVa',
       '#NoAlComunismoEnElPeru', '#NoAlTerrorismo', '#NoAlComunismo',
       '#LosBuenosSomosMas', '#pedrocastilloNICagando',
       '#PedroTiraPiedras', '#Genocidas', '#KeikoFujimori',
       '#CastilloMachistayHomofobico', '#CastilloYCerronUnSoloCorazon',
       '#EleccionesPerú2021', '#DefiendeTuVoto', '#RespetaMiVoto',
       '#LaMaldiciondelaSenoraK', '#LaCamisetaNoSeMancha',
       '#FujimorismoNuncaMás', '#SegundaVuelta',
       '#EleccionesBicentenario', '#Elecciones2021PE', '#PeruDecide2021',
       '#PedroCastillo', '#Elecciones2021', '#eleccionesperu', '#ONPE',
       '#EleccionesPeru', '#Elecciones2021Peru', '#Onpe', 'ONPE', '#JNE',
       '#ONPEPeru', '#JNEPeru', '#PeruLibre', '#PERULIBRE',
       '#TerrorismoNuncaMas', '#Jne', '#PedroCastilloTe', '#JNE_Peru',
       '#CIDH', '#APensarMás', '#ParoEnCusco','#ÚLTIMAHORA', '#PuntoFinal', '#ReformaPolitica',
       '#ReformaElectoral', '#CastillorenunciaYA','#SucesiónPresidencial', '#NuevasReglas', '#NuevasElecciones',
       '#QueSeVayanTodos', '#LagartoVizcarra', '#TC','#Legislativo', '#Ejecutivo', '#RenunciaCastillo', '#EstadoCombi',
       '#CierrenElCongreso', '#CongresoVerguenzaNacional', '#Mafiosos',
       '#Referendum','#AsambleaConstituyente', '#ReformaPolíticaYA',
       '#burrocastillo', '#CerronismoNuncaMas', '#congresomiserable', '#FueraCongreso','#Anibal','#AnibalTorres','#Bellido',
          '#Cerron', '#Congreso','#gabinete', '#MariadelCarmenAlva','#Mirtha','#MirthaVasquez','#premier', '#VacanciaPedroCastillo',
           '#VacanciaPedroCastilloYa']

In [67]:
#78
tag_vac = ['#VacúnateYa', '#COVID19', '#PongoElHombro', '#LaVacunaEsVida',
       '#vacuna', '#vacunaton', '#NoBajemosLaGuardia',
       '#VamosaSalirAdelante', '#vacunacion', '#sialavacuna',
       '#VacunaFest', '#TodasLasVacunasSirven', '#VacunaCOVID19',
       '#antivacunas', '#NuevaCepa', '#Sinopharm', '#YoApoyoaBeto',
       '#ErnestoBustamante', '#EsSaludPeru', '#pandemia', '#covid',
       '#covid_19', '#SagastiGenocida', '#SuizaLab', '#AstraZeneca',
       '#Vacunate', '#Vacunagate', '#Minsa', '#Pfizer', '#delta',
       '#variante', '#PfizerBiontech', '#Moderna', '#VacunacionPeru',
       '#VacunateYa', '#Minsa_Peru', '#Omicron',
       '#PreparadosSinSalirDeCasa', '#yomequedoencasa', '#TomaNota',
       '#SARSCoV2', '#Cubrebocas', '#MeCuido', '#PorAmorALaVida',
       '#confinamiento', '#SaludEnTuVIda', '#mascarillas','#MedidasPreventivas', '#TeCuidas',
       '#LavadoDeManos', '#UsoDeCubrebocas', '#SanaDistancia',
       '#PrevenirEsSalud', '#ventilación', '#distancia', '#hidrogel','#NoAlBozal',
       '#CuidarteEsCuidarnos', '#SaludMental',
       '#oxímetro', '#DispositivosMédicos', '#Oxigenación',
       '#teletrabajo', '#LongCovid', '#EstornudoDeEtiqueta',
       '#PCR ', '#emergencia', '#Tapabocas']

In [5]:
def Crear_carpeta(Rutabase, name_carpeta):
    # Se define el nombre de la carpeta o directorio a crear
    directorio = Rutabase + name_carpeta
    if os.path.exists(directorio):
        shutil.rmtree(directorio)
        os.makedirs(directorio)
        print("Se reemplazo el directorio %s " % directorio)
    else:
        try:
            os.mkdir(directorio)
        except OSError:
            print("La creación del directorio %s falló" % directorio)
        else:
            print("Se ha creado el directorio: %s " % directorio)

In [6]:
def Buscar_Palabra(cadena, palabra_incluida):
    lista_cadenas = [cadena]
    #print(cadena.find(palabra_incluida))
    if cadena.find(palabra_incluida) < 0:
        lista_cadenas.append(palabra_incluida)
    return lista_cadenas
    

In [7]:
def search_tweets(i, since, until):
    print('entre', i)
    tag = Buscar_Palabra(i,'Peru')
    c = twint.Config()
    print(tag)
    c.Search = tag
    c.Language = 'es'
    c.Since = since
    c.Until = until
    c.Limit = 10000
    c.Pandas = True
    c.Hide_output= True
    twint.run.Search(c)
    df_twint = twint.storage.panda.Tweets_df
    return df_twint
    

In [8]:
def Generar_Tweets(lista, Rutabase, since, until):
    lista = pd.unique(lista)
    numero_tweets = 0
    for i in lista:
        sub_df = pd.DataFrame()
        #print(i)
        j = 0
        #sub_df = search_tweets(i)
        while len(sub_df)<=5 & (j<6) :
            sub_df = search_tweets(i, since, until)
            j = j + 1
            #print(j)
            #print(len(sub_df))
        Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        #print(sub_df)
        sub_df.to_csv(Rutasol, index = False)
        #df_twint = Eliminar_Repetidos(df_twint, 'tweet', 'spanish', 0.5)
        #print(df_twint)
        #print(len(df_twint))
        #numero_tweets = numero_tweets + len(sub_df)
        
        #print(numero_tweets)
        print('Se ha creado correctamente el ' + i + '.csv, recuperando ' + str(len(sub_df)) + ' tweets')
    #df_Final.to_csv("Tweets_prueba.csv", index=False)

In [9]:
def SeleccionarTweets(lista, Rutabase):
    total_df = pd.DataFrame() 
    count = 0
    for i in lista:
        Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        #print(Rutasol)
        df = pd.read_csv(Rutasol, delimiter=',')
        #print(df)
        new_df = df[df['language'] == 'es']
        print('Se ha recuperado ' + str(len(new_df)) + ' tweets en el idioma espanhol')
        new_df = new_df.reset_index(drop = True)
        new_df = Eliminar_Repetidos(new_df, 'tweet', 'spanish', 0.5)
        print('Se han eliminado los repetidos del archivo '+ i +'.csv'+ ' , el nuevo numero de tweets es ' + str(len(new_df)))
        #count = count + len(df)
        total_df = pd.concat([total_df, new_df],ignore_index=False)
    #print(total_df.shape)
    total_df = total_df.reset_index(drop = True)
    print('Se ha recuperado ' + str(len(total_df)) + ' tweets en total' )
    return total_df
   
#def Eliminar_Reptidos_Archivo(df):
    
#df[(df['date'] > '2021-04-01') & (df['date'] < '2021-10-01')]

In [10]:
def Recuperar_Indice(lista_indices):
    index = []
    for i in lista_indices:
        index.append(i[1])
    index = pd.unique(index)
    index = list(index)
    return index

In [11]:
def Eliminar_Repetidos(df, label, language, threshold):
    tfidf = TfidfVectorizer(stop_words = stopwords.words(language))
    # Construct the TF-IDF matrix
    tfidf_matrix = tfidf.fit_transform(df[label])
    #print(shape(tfidf_matrix))
    # Generate the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix[:20000])
    #print(cosine_sim)
    #Lista que recupera los tweets
    A = []
    for i in range(0,len(cosine_sim)):
        for k in range(0,len(cosine_sim)):
            if ((k>i) and (cosine_sim[i][k] >= threshold)):
                #print(k)
                A.append([i,k])
                
    indices_tweets = Recuperar_Indice(A)
    df = df.drop(indices_tweets)
    #print(df)
    df = df.reset_index(drop = True)
    print('Se han recuperado ' + str(len(df)) + ' tweets')
    return df
    

In [76]:
#Rutabase = r'C:/Users/USER/Documents/Git/SATwint/'
Rutabase =  r'C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/'
#carpeta = 'data4/'
def Obtener_dataset(lista_tweet, name_carpeta, label, language, name):
    #Crear_carpeta(Rutabase, name_carpeta)
    Ruta = Rutabase + name_carpeta
    #Generar_Tweets(lista_tweet, Ruta,'2020-01-01', '2022-04-01')
    df = SeleccionarTweets(lista_tweet, Ruta)
    #print(df)
    df = Eliminar_Repetidos(df, label, language, 0.5)
    df.to_csv(name + '.csv')
    return df

In [79]:
#final_df = Obtener_dataset(tag_pol_3,'Eleccion/', 'tweet', 'spanish', 'df_vacunacion' )
final_df = Obtener_dataset(tag_pol,'Elecciones/', 'tweet', 'spanish', 'df_elecciones' )

Se ha recuperado 37 tweets en el idioma espanhol
Se han recuperado 33 tweets
Se han eliminado los repetidos del archivo #KeikoNova.csv , el nuevo numero de tweets es 33
Se ha recuperado 11 tweets en el idioma espanhol
Se han recuperado 11 tweets
Se han eliminado los repetidos del archivo #keikonuncamas.csv , el nuevo numero de tweets es 11
Se ha recuperado 18 tweets en el idioma espanhol
Se han recuperado 17 tweets
Se han eliminado los repetidos del archivo #FujimoriNuncaMás.csv , el nuevo numero de tweets es 17
Se ha recuperado 16 tweets en el idioma espanhol
Se han recuperado 16 tweets
Se han eliminado los repetidos del archivo #NoAKeiko.csv , el nuevo numero de tweets es 16
Se ha recuperado 57 tweets en el idioma espanhol
Se han recuperado 54 tweets
Se han eliminado los repetidos del archivo #Keikino.csv , el nuevo numero de tweets es 54
Se ha recuperado 20 tweets en el idioma espanhol
Se han recuperado 19 tweets
Se han eliminado los repetidos del archivo #KeikoTAPERdiendo.csv , el 

Se han recuperado 70 tweets
Se han eliminado los repetidos del archivo ONPE.csv , el nuevo numero de tweets es 70
Se ha recuperado 18 tweets en el idioma espanhol
Se han recuperado 15 tweets
Se han eliminado los repetidos del archivo #JNE.csv , el nuevo numero de tweets es 15
Se ha recuperado 9 tweets en el idioma espanhol
Se han recuperado 9 tweets
Se han eliminado los repetidos del archivo #ONPEPeru.csv , el nuevo numero de tweets es 9
Se ha recuperado 17 tweets en el idioma espanhol
Se han recuperado 14 tweets
Se han eliminado los repetidos del archivo #JNEPeru.csv , el nuevo numero de tweets es 14
Se ha recuperado 53 tweets en el idioma espanhol
Se han recuperado 41 tweets
Se han eliminado los repetidos del archivo #PeruLibre.csv , el nuevo numero de tweets es 41
Se ha recuperado 53 tweets en el idioma espanhol
Se han recuperado 41 tweets
Se han eliminado los repetidos del archivo #PERULIBRE.csv , el nuevo numero de tweets es 41
Se ha recuperado 20 tweets en el idioma espanhol
Se h

Se han recuperado 1613 tweets


In [80]:
final_df

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1508817563714740232,1507719027996516354,1.648565e+12,2022-03-29 11:45:14,-300,NaN,"@CIDH @CorteIDH #Peru En investigaciones, el n...",es,"['peru', 'fujimorismonuncamas', 'keikonova']",[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'CIDH', 'name': 'CIDH - IACHR...",NaN,NaN,NaN,NaN
1,1506255176885325826,1506021954557415425,1.647954e+12,2022-03-22 10:03:13,-300,NaN,"@GuilleBermejo01 el ex-presidente Sagasti es, ...",es,['keikonova'],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'GuilleBermejo01', 'name': 'G...",NaN,NaN,NaN,NaN
2,1504796037696958469,1504796037696958469,1.647606e+12,2022-03-18 09:25:07,-300,NaN,Perú siempre peleando la misma batalla contra ...,es,"['fujimorigenocida', 'fujimorinuncamas', 'keik...",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1496245966952976385,1496245966952976385,1.645568e+12,2022-02-22 19:10:12,-300,NaN,Vote x Castillo en la segunda vuelta ante la a...,es,"['narcoestado', 'keikonova']",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1481936501327998977,1481648281755078657,1.642156e+12,2022-01-14 07:29:29,-300,NaN,@willaxtv Ese es el pensamiento fujimorista y ...,es,"['fujimorismonuncamas', 'noalamafiafujimorista...",[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'willaxtv', 'name': 'Willax T...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,1460385159426019329,1460385159426019329,1.637018e+12,2021-11-15 20:12:09,-300,NaN,Castillo gobierna el Perú con sus amigazos de ...,es,['vacanciapredrocastilloya'],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1609,1460353340068483074,1460353340068483074,1.637010e+12,2021-11-15 18:05:42,-300,NaN,Perú Libre escenifica una lucha interna fictic...,es,"['vacanciapresidencialya', 'vacanciapredrocast...",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1610,1460301787693568008,1460290058217566209,1.636998e+12,2021-11-15 14:40:51,-300,NaN,@SUTEP_Peru @gabrielalpp ¿Qué esperan que no s...,es,['vacanciapredrocastilloya'],[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'SUTEP_Peru', 'name': 'SUTEP'...",NaN,NaN,NaN,NaN
1611,1460298705085546501,1460298705085546501,1.636997e+12,2021-11-15 14:28:36,-300,NaN,Eres o no!! lo que queremos es que se vayan to...,es,"['vacanciapresidencialya', 'vacanciapredrocast...",[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
